In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fastapi uvicorn pyngrok streamlit nest_asyncio sqlalchemy passlib   --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.3 MB/s eta 0:00:00


In [3]:
from pyngrok import ngrok
ngrok.set_auth_token("30sSGKyp7t79yoNzHAynoOyIOR5_3tM76e6qet58uq6Tbfzhv")

In [4]:
import sqlite3
from passlib.context import CryptContext

DB_PATH = "/content/drive/MyDrive/students.db"
  # keep your local path

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")


conn = sqlite3.connect(DB_PATH)
c = conn.cursor()

# 1. Users Table (Existing)
c.execute("""
CREATE TABLE IF NOT EXISTS users (
 id INTEGER PRIMARY KEY AUTOINCREMENT,
 name TEXT NOT NULL,
 email TEXT UNIQUE NOT NULL,
 password TEXT NOT NULL,  -- CRITICAL: MUST be TEXT
 role TEXT NOT NULL CHECK(role IN ('student', 'admin'))
);
""")

# 2. Study Log Table (Existing)
c.execute("""
CREATE TABLE IF NOT EXISTS study_log (
 id INTEGER PRIMARY KEY AUTOINCREMENT,
 user_id INTEGER NOT NULL,
 date DATETIME DEFAULT CURRENT_TIMESTAMP,
 study_hours REAL NOT NULL,
 social_media_hours REAL NOT NULL,
 sleep_hours REAL NOT NULL,
 stress_level INTEGER NOT NULL,
 exam_score REAL,
 notes TEXT,
 FOREIGN KEY (user_id) REFERENCES users(id)
)
""")

# 3. NEW: Password Resets Table for OTP
c.execute("""
CREATE TABLE IF NOT EXISTS password_resets (
 id INTEGER PRIMARY KEY AUTOINCREMENT,
 user_id INTEGER NOT NULL,
 otp TEXT NOT NULL,
 expires_at INTEGER NOT NULL,
 FOREIGN KEY (user_id) REFERENCES users(id)
)
""")


# default admin
c.execute("SELECT * FROM users WHERE email = 'admin@gmail.com'")
if not c.fetchone():
    hashed = pwd_context.hash("admin123")
    c.execute(
        "INSERT INTO users (name, email, password, role) VALUES (?, ?, ?, ?)",
        ("Admin", "admin@gmail.com", hashed, "admin"),
    )
    print("Default Admin Created → admin@gmail.com / admin123")

conn.commit()
conn.close()
print("Database ready!")

Database ready!


In [5]:

%%writefile backend.py
from fastapi import FastAPI, HTTPException, Header, Depends
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import Optional, List
import sqlite3
from passlib.context import CryptContext
import re
import time
import random
import joblib
import numpy as np
import pandas as pd
from datetime import datetime



# Removed: smtplib and email.message imports for terminal mock method

# ======================================================================
# --- CONFIGURATION ---
# ======================================================================

DB_PATH = "/content/drive/MyDrive/students.db" # Change if needed
MODEL_PATH = "/content/drive/MyDrive/student_cluster_model.pkl"
SCALER_PATH = "/content/drive/MyDrive/student_scaler.pkl"

# ======================================================================
# --- FastAPI Setup ---
# ======================================================================
app = FastAPI()
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
# ======================================================================
# --- Load K-Means Model & Scaler
# ======================================================================
try:
    kmeans = joblib.load(MODEL_PATH)
    scaler = joblib.load(SCALER_PATH)
    ML_AVAILABLE = True
    print("AI Clustering Model Loaded Successfully!")
except Exception as e:
    print(f"Model load failed: {e} → Using rule-based fallback")
    kmeans = None
    scaler = None
    ML_AVAILABLE = False

# ======================================================================
# --- Cluster Names & Recommendations
# ======================================================================
CLUSTER_INFO = {
    0: {
        "name": "Average",
        "title": "Time to Level Up Your Study Game!",
        "tips": [
            "Add just 30 extra minutes of focused study every day",
    "Cut social media by 15 minutes daily",
    "Do one “Power Hour” on weekends",
    "Review notes for 10 minutes before bed",
    "Walk or stretch for 5 minutes every study hour",
    "Celebrate small wins every Friday",
    "Keep logging daily—it only takes 30 seconds",
    "Tell one friend about your goal"
        ]
    },
    1: {
        "name": "Stressed Over-Achiever",
        "title": " You're Burning Out",
        "color": "#EF4444",
        "emoji": "Fire + Tired Face",
        "priority": "CRITICAL",
        "tips": [
            "Use Pomodoro (25 min study + 5 min break)",
                    "Add 10-min daily meditation",
                    "Protect sleep (7.5h minimum)",
                    "Take regular breaks every 30 minutes during study sessions.",
                    "Consider exercise or yoga to manage your high stress levels.",
                    "Review your study material before bed instead of right before exams."
        ]
    },
    2: {
        "name": "Balanced Top Performer",
        "title": "You're Doing Everything Right — Keep Going!",
        "tips": [
                    "Continue Same study routine + 7–8h sleep + minimal distractions",
                    "Maintain your current study routine - it's working perfectly.",
                    "Ensure 7-8 hours of sleep nightly for optimal memory consolidation.",
                    "Balance study and leisure effectively to avoid burnout.",
                    "Keep managing stress well - your low stress levels are a strength.",
                    "Track your progress weekly and celebrate your achievements."
                ]
    },
    3: {
        "name": "Distracted",
        "title": "Your Brain is Brilliant — Stop Scrolling!",
        "tips": [
            "Block Instagram/TikTok during study hours (use Freedom or Cold Turkey app)",
                    "Study with phone in another room",
                    "Use app blockers to prevent access to social media apps.",
                    "Increase study hours gradually - aim for 1-2 more hours per week.",
                    "Create a dedicated study space free from distractions and notifications.",
                    "Set specific study goals for each session to stay motivated and focused."
        ]
    }
}

# ======================================================================
# ---- Helper Functions ----
# ======================================================================

def valid_email(email: str) -> bool:
    """Basic email format validation."""
    return re.match(r"^[\w\.\-]+@[\w\.\-]+\.\w+$", email) is not None

def generate_otp():
    """Generates a 6-digit OTP."""
    return str(random.randint(100000, 999999))



def send_otp_terminal(receiver_email: str, otp: str):
    """
    MOCK SENDER: Outputs the OTP directly to the terminal for development.
    This replaces the real email sending logic for local testing.
    """
    print("======================================================")
    print(f"** MOCK OTP SENT (FOR TESTING) **")
    print(f"TARGET: {receiver_email}")
    print(f"OTP CODE: {otp}")
    print("======================================================")
    return True


# ---- Auth helper ----

def get_current_user(x_user_id: Optional[str] = Header(None)):
    if not x_user_id:
        raise HTTPException(401, "Missing user ID")
    try:
        return int(x_user_id)
    except Exception:
        raise HTTPException(401, "Invalid user ID")


# ======================================================================
# ---- Pydantic models ----
# ======================================================================
class UserIn(BaseModel):
    name: str
    email: str
    password: str
    role: str  # "student" or "admin"


class LoginIn(BaseModel):
    email: str
    password: str
    role: str


class StudyLogIn(BaseModel):
    study_hours: float
    social_media_hours: float
    sleep_hours: float
    stress_level: int
    exam_score: Optional[float] = None
    notes: Optional[str] = None



class ResetPasswordIn(BaseModel):
    user_id: int # user_id obtained from the successful /verify-otp call
    new_password: str


# ======================================================================
# ---- ENDPOINTS ----
# ======================================================================

@app.post("/signup")
def signup(u: UserIn):
    if u.role not in ["student", "admin"]:
        raise HTTPException(400, "Invalid role")
    if not valid_email(u.email):
        raise HTTPException(400, "Invalid email")

    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT id FROM users WHERE email = ?", (u.email,))
    if cur.fetchone():
        conn.close()
        raise HTTPException(400, "Email already registered")

    hashed = pwd_context.hash(u.password)
    cur.execute(
        "INSERT INTO users (name, email, password, role) VALUES (?, ?, ?, ?)",
        (u.name, u.email, hashed, u.role),
    )
    conn.commit()
    conn.close()
    return {"message": "Signup successful"}




@app.post("/login")
def login(u: LoginIn):
    if not valid_email(u.email):
        raise HTTPException(400, "Invalid email")

    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute(
        "SELECT id, name, email, password, role FROM users WHERE email = ? AND role = ?",
        (u.email, u.role),
    )
    user = cur.fetchone()
    conn.close()

    if not user or not pwd_context.verify(u.password, user[3]):
        raise HTTPException(401, "Wrong credentials")

    return {
        "message": "Login OK",
        "user": {"id": user[0], "name": user[1], "email": user[2], "role": user[4]},
    }


@app.get("/students")
def get_students():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT id, name, email FROM users WHERE role = 'student'")
    rows = cur.fetchall()
    conn.close()
    return [{"id": r[0], "name": r[1], "email": r[2]} for r in rows]


@app.post("/add-user")
def add_user(u: UserIn):
    if not valid_email(u.email):
        raise HTTPException(400, "Invalid email")

    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT id FROM users WHERE email = ?", (u.email,))
    if cur.fetchone():
        conn.close()
        raise HTTPException(400, "Email already exists")

    hashed = pwd_context.hash(u.password)
    cur.execute(
        "INSERT INTO users (name, email, password, role) VALUES (?, ?, ?, ?)",
        (u.name, u.email, hashed, u.role),
    )
    conn.commit()
    conn.close()
    return {"message": "User added"}

def compute_cluster(user_id: int) -> Optional[int]:
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # Validate student exists
    cur.execute("SELECT name FROM users WHERE id = ? AND role = 'student'", (user_id,))
    user = cur.fetchone()
    if not user:
        conn.close()
        return None

    # Get last 10 logs
    cur.execute("""
        SELECT study_hours, social_media_hours, sleep_hours, stress_level, exam_score
        FROM study_log
        WHERE user_id = ?
        ORDER BY date DESC, id DESC
        LIMIT 10
    """, (user_id,))
    rows = cur.fetchall()
    conn.close()

    if len(rows) == 0:
        return None

    # Convert to DataFrame and compute averages
    df = pd.DataFrame(rows, columns=["study_hours", "social_media_hours", "sleep_hours", "stress_level", "exam_score"])
    df["exam_score"] = pd.to_numeric(df["exam_score"], errors='coerce')

    averages = {
        "study_hours": round(df["study_hours"].mean(), 1),
        "social_media_hours": round(df["social_media_hours"].mean(), 1),
        "sleep_hours": round(df["sleep_hours"].mean(), 1),
        "stress_level": round(df["stress_level"].mean(), 1),
        "exam_score": round(df["exam_score"].mean(), 1) if not df["exam_score"].isna().all() else 70
    }

    s, sm, sl, st, sc = averages.values()

    # --- Use K-Means Model if available ---
    if ML_AVAILABLE:
        try:
            feature_vector = np.array([[s, sm, sl, st, sc]])
            scaled = scaler.transform(feature_vector)
            cluster_id = int(kmeans.predict(scaled)[0])
        except Exception:
            cluster_id = 2  # default to balanced
    else:
        # Simple rule-based fallback
        if s >= 4 and sc >= 80 and sm <= 2.5 and st <= 4.5:
            cluster_id = 2  # Balanced top performer
        elif s >= 4 and sc >= 70 and st >= 6:
            cluster_id = 1  # Stressed
        elif sm >= 3 or st <= 4 and sc <= 65:
            cluster_id = 3  # Distracted
        else:
            cluster_id = 0  # Average

    return cluster_id

@app.post("/add-study-log")
def add_study_log(log: StudyLogIn, current_user: int = Depends(get_current_user)):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT role FROM users WHERE id = ?", (current_user,))
    user = cur.fetchone()
    if not user or user[0] != "student":
        conn.close()
        raise HTTPException(403, "Only students can add logs")

    cur.execute(
        """
        INSERT INTO study_log
        (user_id, study_hours, social_media_hours, sleep_hours, stress_level, exam_score, notes)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """,
        (
            current_user,
            log.study_hours,
            log.social_media_hours,
            log.sleep_hours,
            log.stress_level,
            log.exam_score,
            log.notes,
        ),
    )
    conn.commit()


    # NEW: Compute and update cluster after adding log
    cluster = compute_cluster(current_user)
    if cluster is not None:
        cur.execute(
            "INSERT OR REPLACE INTO user_clusters (user_id, cluster_id) VALUES (?, ?)",
            (current_user, cluster)
        )
        conn.commit()

    conn.close()
    return {"message": "Study log added successfully"}


@app.get("/study-logs/{user_id}")
def get_study_logs(user_id: int):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT id FROM users WHERE id = ? AND role = 'student'", (user_id,))
    if not cur.fetchone():
        conn.close()
        raise HTTPException(400, "Invalid student")

    cur.execute(
        """
        SELECT id, date, study_hours, social_media_hours, sleep_hours,
               stress_level, exam_score, notes
        FROM study_log
        WHERE user_id = ?
        ORDER BY date DESC
        """,
        (user_id,),
    )
    rows = cur.fetchall()
    conn.close()

    return [
        {
            "id": r[0],
            "date": str(r[1]),
            "study_hours": r[2],
            "social_media_hours": r[3],
            "sleep_hours": r[4],
            "stress_level": r[5],
            "exam_score": r[6],
            "notes": r[7],
        }
        for r in rows
    ]

@app.get("/admin/all-study-logs")
def get_all_study_logs(current_user: int = Depends(get_current_user)):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # Check if the requesting user is an admin
    cur.execute("SELECT role FROM users WHERE id = ?", (current_user,))
    user = cur.fetchone()
    if not user or user[0] != "admin":
        conn.close()
        raise HTTPException(403, "Access denied. Admin privileges required.")

    # Fetch all logs along with student name and email
    cur.execute(
        """
        SELECT
            sl.id,
            sl.date,
            sl.study_hours,
            sl.social_media_hours,
            sl.sleep_hours,
            sl.stress_level,
            sl.exam_score,
            sl.notes,
            u.name AS student_name,
            u.email AS student_email
        FROM study_log sl
        JOIN users u ON sl.user_id = u.id
        ORDER BY sl.date DESC
        """,
    )
    rows = cur.fetchall()
    conn.close()

    # Map the results to a list of dictionaries
    return [
        {
            "log_id": r[0],
            "date": str(r[1]),
            "study_hours": r[2],
            "social_media_hours": r[3],
            "sleep_hours": r[4],
            "stress_level": r[5],
            "exam_score": r[6],
            "notes": r[7],
            "student_name": r[8],
            "student_email": r[9],
        }
        for r in rows
    ]




# --- IMPORTANT: Change this block in your backend.py ---

# --- Replace or Verify this function in your backend.py ---

# --- Replace or Verify this function in your backend.py ---

@app.post("/reset-password")
def reset_password(r: ResetPasswordIn):
    conn = None # Initialize conn outside try to ensure finally block can access it

    try:
        conn = sqlite3.connect(DB_PATH)
        cur = conn.cursor()

        # 1. Input Validation (Ensure user_id is valid and exists)
        try:
            user_id_int = int(r.user_id)
        except (TypeError, ValueError):
            raise HTTPException(status_code=400, detail="Invalid user ID data type.")

        cur.execute("SELECT id FROM users WHERE id = ?", (user_id_int,))
        if not cur.fetchone():
            raise HTTPException(status_code=400, detail="User not found.")

        # 2. Hash and Update Password (SUCCESSFUL PART)
        hashed_password = pwd_context.hash(r.new_password)

        cur.execute(
            "UPDATE users SET password = ? WHERE id = ?",
            (hashed_password, user_id_int),
        )
        conn.commit() # The password is changed HERE

        # 3. Clean up OTPs
        cur.execute("DELETE FROM password_resets WHERE user_id = ?", (user_id_int,))
        conn.commit()

        # 4. Successful return
        return {"message": "Password updated successfully. You can now log in."}

    except sqlite3.Error as e:
        # Rollback in case of database error
        if conn:
            conn.rollback()
        # Return a specific error detail if the database failed *before* commit
        raise HTTPException(status_code=500, detail=f"Database error during update: {e}")

    finally:
        # CRITICAL: This ensures the connection is closed even if the function fails
        if conn:
            conn.close()

# --- Add this new function to your backend.py ---

class LogUpdate(BaseModel):
    log_id: int
    study_hours: float
    social_media_hours: float
    sleep_hours: float
    stress_level: int
    exam_score: Optional[float] = None
    notes: Optional[str] = None

class UpdateNameIn(BaseModel):
    new_name: str

@app.post("/update-name")
def update_name(data: UpdateNameIn, current_user: int = Depends(get_current_user)):
    if not data.new_name.strip():
        raise HTTPException(400, "Name cannot be empty")

    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # Update the name
    cur.execute("UPDATE users SET name = ? WHERE id = ?", (data.new_name.strip(), current_user))

    if cur.rowcount == 0:
        conn.close()
        raise HTTPException(404, "User not found")

    conn.commit()
    conn.close()

    return {"message": "Name updated successfully"}


@app.post("/admin/update-study-log")
def update_study_log(log_data: LogUpdate, current_user: int = Depends(get_current_user)):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # 1. Check Admin Privilege
    cur.execute("SELECT role FROM users WHERE id = ?", (current_user,))
    user_role = cur.fetchone()
    if not user_role or user_role[0] != "admin":
        conn.close()
        raise HTTPException(status_code=403, detail="Access denied. Admin privileges required.")

    # 2. Check if log exists
    cur.execute("SELECT id FROM study_log WHERE id = ?", (log_data.log_id,))
    if not cur.fetchone():
        conn.close()
        raise HTTPException(status_code=404, detail=f"Study log ID {log_data.log_id} not found.")

    # 3. Perform Update
    cur.execute(
        """
        UPDATE study_log SET
            study_hours = ?,
            social_media_hours = ?,
            sleep_hours = ?,
            stress_level = ?,
            exam_score = ?,
            notes = ?
        WHERE id = ?
        """,
        (
            log_data.study_hours,
            log_data.social_media_hours,
            log_data.sleep_hours,
            log_data.stress_level,
            log_data.exam_score,
            log_data.notes,
            log_data.log_id,
        ),
    )
    conn.commit()
    conn.close()
    return {"message": f"Study log ID {log_data.log_id} updated successfully."}

# --- Add this new function to your backend.py ---

class LogDelete(BaseModel):
    log_id: int

@app.post("/admin/delete-study-log")
def delete_study_log(log_data: LogDelete, current_user: int = Depends(get_current_user)):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # 1. Check Admin Privilege
    cur.execute("SELECT role FROM users WHERE id = ?", (current_user,))
    user_role = cur.fetchone()
    if not user_role or user_role[0] != "admin":
        conn.close()
        raise HTTPException(status_code=403, detail="Access denied. Admin privileges required.")

    # 2. Perform Delete
    cur.execute("DELETE FROM study_log WHERE id = ?", (log_data.log_id,))

    if cur.rowcount == 0:
        conn.close()
        raise HTTPException(status_code=404, detail=f"Study log ID {log_data.log_id} not found.")

    conn.commit()
    conn.close()
    return {"message": f"Study log ID {log_data.log_id} deleted successfully."}

@app.get("/recommendation/{user_id}")
def get_recommendation(user_id: int):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # Validate student exists
    cur.execute("SELECT name FROM users WHERE id = ? AND role = 'student'", (user_id,))
    user = cur.fetchone()
    if not user:
        conn.close()
        raise HTTPException(404, "Student not found")
    student_name = user[0]

    # Get last 10 logs
    cur.execute("""
        SELECT study_hours, social_media_hours, sleep_hours, stress_level, exam_score
        FROM study_log
        WHERE user_id = ?
        ORDER BY date DESC, id DESC
        LIMIT 10
    """, (user_id,))
    rows = cur.fetchall()
    conn.close()

    if len(rows) == 0:
        return {
            "cluster_name": "Welcome! Start Logging",
            "title": "Your AI Coach is Ready",
            "color": "#6366F1",
            "emoji": "Waving Hand",
            "priority": "INFO",
            "tips": ["Log your first study session to unlock your personalized AI coach!"],
            "logs_used": 0,
            "averages": {}
        }

    # Convert to DataFrame and compute averages
    df = pd.DataFrame(rows, columns=["study_hours", "social_media_hours", "sleep_hours", "stress_level", "exam_score"])
    df["exam_score"] = pd.to_numeric(df["exam_score"], errors='coerce')

    averages = {
        "study_hours": round(df["study_hours"].mean(), 1),
        "social_media_hours": round(df["social_media_hours"].mean(), 1),
        "sleep_hours": round(df["sleep_hours"].mean(), 1),
        "stress_level": round(df["stress_level"].mean(), 1),
        "exam_score": round(df["exam_score"].mean(), 1) if not df["exam_score"].isna().all() else 70
    }

    s, sm, sl, st, sc = averages.values()

    # --- Use K-Means Model if available ---
    if ML_AVAILABLE:
        try:
            feature_vector = np.array([[s, sm, sl, st, sc]])
            scaled = scaler.transform(feature_vector)
            cluster_id = int(kmeans.predict(scaled)[0])
            info = CLUSTER_INFO[cluster_id]
            model_used = "K-Means AI"
        except Exception as e:
            print(f"ML failed: {e} → fallback to rules")
            info = CLUSTER_INFO[2]  # default to balanced
            model_used = "Rule-Based Fallback"
    else:
        # Simple rule-based fallback (you can improve this later)
        if s >=4 and sc>=80 and sm<=2.5 and st<=4.5:
            info = CLUSTER_INFO[2]  # Balanced top perfoemer
        elif s >=4 and sc>=70 and st>=6:
            info = CLUSTER_INFO[1]  # Stressed
        elif sm>=3 or st<=4 and sc<=65:
            info = CLUSTER_INFO[3]  # Distracted
        else:
            info = CLUSTER_INFO[0]  # Average
        model_used = "Rule-Based"

    return {
        "user_id": user_id,
        "user_name": student_name,
        "cluster_name": info["name"],
        "title": info["title"],
        "tips": info["tips"],
        "averages": averages,
        "logs_used": len(rows),
        "model_used": model_used,
        "confidence": 98 if ML_AVAILABLE else 100,
        "updated_at": datetime.now().strftime("%b %d, %Y")
    }

@app.get("/admin/clusters")
def get_cluster_distribution(current_user: int = Depends(get_current_user)):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # Check if user is admin - FIXED: only one fetchone()
    cur.execute("SELECT role FROM users WHERE id = ?", (current_user,))
    user = cur.fetchone()
    if not user or user[0] != "admin":
        conn.close()
        raise HTTPException(status_code=403, detail="Admin access required")

    # Get cluster distribution
    cur.execute("""
        SELECT uc.cluster_id, COUNT(*) as count
        FROM user_clusters uc
        GROUP BY uc.cluster_id
        ORDER BY uc.cluster_id
    """)
    rows = cur.fetchall()
    conn.close()

    # Initialize all clusters to 0
    distribution = {0: 0, 1: 0, 2: 0, 3: 0}
    for cluster_id, count in rows:
        distribution[cluster_id] = count

    cluster_names = {
        0: "Average",
        1: "Stressed Over-Achiever",
        2: "Balanced Top Performer",
        3: "Distracted"
    }

    return {
        "distribution": distribution,
        "total_clustered": sum(distribution.values()),
        "names": cluster_names
    }

@app.get("/student-cluster/{user_id}")
def get_student_cluster(user_id: int):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # Check if user exists and is a student
    cur.execute("SELECT name FROM users WHERE id = ? AND role = 'student'", (user_id,))
    user = cur.fetchone()
    if not user:
        conn.close()
        raise HTTPException(404, "Student not found")

    student_name = user[0]

    # Get the latest cluster (order by updated_at or id DESC)
    cur.execute("""
        SELECT cluster_id
        FROM user_clusters
        WHERE user_id = ?
        LIMIT 1
    """, (user_id,))

    cluster_row = cur.fetchone()
    conn.close()

    if not cluster_row:
        return {
            "user_id": user_id,
            "name": student_name,
            "cluster_id": None,
            "cluster_name": "Not yet clustered (needs at least one log)"
        }

    cluster_id = cluster_row[0]
    cluster_name = CLUSTER_INFO.get(cluster_id, {}).get("name", "Unknown Cluster")

    return {
        "user_id": user_id,
        "name": student_name,
        "cluster_id": cluster_id,
        "cluster_name": cluster_name
    }

Writing backend.py


In [6]:
import nest_asyncio, uvicorn, threading
from pyngrok import ngrok

nest_asyncio.apply()
ngrok.kill()

public_url = ngrok.connect(8000)
print(" FastAPI Public URL:", public_url.public_url)

def run_backend():
    uvicorn.run("backend:app", host="0.0.0.0", port=8000)

threading.Thread(target=run_backend).start()

 FastAPI Public URL: https://1ac2-34-26-222-18.ngrok-free.app


In [7]:
!mkdir -p .streamlit

In [8]:
%%writefile .streamlit/config.toml
[theme]
base="light"
primaryColor="#E63946"
backgroundColor="#F54927"
secondaryBackgroundColor="#A8DADC"
textColor="#1D3557

Writing .streamlit/config.toml


In [9]:
%%writefile frontend.py
import streamlit as st
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import time
from typing import Optional

if "ngrok_url" in st.session_state:
    BACKEND_URL = st.session_state.ngrok_url
else:
    BACKEND_URL = "http://localhost:8000"

# ---- Initial Configuration & Styling ----
st.set_page_config(
    page_title="StudyTrack AI",
    layout="wide"
)

st.markdown(
    """    <style>
    /* Global Fade-in Animation */
    .main {
        background-color: #79e366;
        animation: fadein 1.5s;
    }
    @keyframes fadein {
        from { opacity: 0; transform: translateY(5px); }
        to { opacity: 1; transform: translateY(0px); }
    }

    /* Button Styling */
    .stButton>button {
        border-radius: 8px;
        padding: 0.6rem 1.2rem;
        font-weight: 700;
        transition: background-color 0.3s;
    }

    /* Primary buttons on landing page */
    .big-primary-btn button {
        background-color: #bec3ee !important;
        font-size: 1.1rem !important;
        padding: 0.8rem 2rem !important;
        border-radius: 12px !important;
    }
    .big-primary-btn button:hover {
        background-color: #e6ed61 !important;
    }

    /* Metric Styling */
    [data-testid="stMetricValue"] {
        font-size: 1.8rem;
        font-weight: 700;
        color: #FF4B4B;
    }

    /* Input field enhancement */
    div.stTextarea label, div.stTextInput label, div.stNumberInput label, div.stSelectbox label {
        font-weight: 600;
        color: #333333;
    }

    /* Info/Success/Error blocks */
    .stAlert {
        border-radius: 8px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }

    /* Active Admin Button */
    .stButton button[data-testid="stButton-primary"] {
        background-color: #FF4B4B;
        color: white;
        border-color: #FF4B4B;
    }
    .stButton button[data-testid="stButton-primary"]:hover {
        background-color: #e34141;
        border-color: #e34141;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

# --- Session State Initialization ---
if "user" not in st.session_state:
    st.session_state.user = None
if "page" not in st.session_state:
    st.session_state.page = "welcome"  # Default to welcome page
if "admin_log_to_update" not in st.session_state:
    st.session_state.admin_log_to_update = None
if "admin_log_data" not in st.session_state:
    st.session_state.admin_log_data = None
if "confirm_delete_id" not in st.session_state:
    st.session_state.confirm_delete_id = None
if "admin_feature_key" not in st.session_state:
    st.session_state.admin_feature_key = "overview"

def safe_date_format(series):
    return pd.to_datetime(series, errors='coerce').dt.strftime("%b %d, %Y • %I:%M %p").fillna("Unknown Date")

# ---- API Helper Function ----
def call(endpoint, data=None, user_id=None):
    r = None
    try:
        headers = {}
        active_user_id = user_id if user_id is not None else (st.session_state.user["id"] if st.session_state.user else None)
        if active_user_id:
            headers["x-user-id"] = str(active_user_id)

        url = f"{BACKEND_URL}{endpoint}"
        if data is None:
            r = requests.get(url, headers=headers, timeout=10)
        else:
            r = requests.post(url, json=data, headers=headers, timeout=10)

        r.raise_for_status()
        try:
            return r.json()
        except ValueError:
            return {"error": "Invalid JSON from server", "detail": r.text}

    except requests.exceptions.RequestException as e:
        try:
            if r is not None:
                try:
                    parsed = r.json()
                    detail = parsed.get("detail") or parsed.get("message") or r.text
                except Exception:
                    detail = r.text
            else:
                detail = str(e)
        except Exception:
            detail = str(e)
        return {"error": "Connection or API Error", "detail": detail}

# ---- Improved Welcome / Landing Page ----
def show_welcome_page():
    st.markdown("<h1 style='text-align: center; color: #FF4B4B; margin-top: 0rem;'>STUDY HABIT RECOMMENDATION SYSTEM</h1>", unsafe_allow_html=True)
    st.markdown("<h2 style='text-align: center; color: #333; margin-bottom: 3rem;'>Your AI-Powered Study Companion</h2>", unsafe_allow_html=True)

    # Hero section with vibrant gradient
    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        st.markdown("""
<div style="
  max-width: 600px;
  width: 100%;
  padding: 3rem;
  margin: 0 auto;
  background: linear-gradient(135deg, #ad4b94 0%, #1A237E 100%);
  border-radius: 24px;
  box-shadow: 0 15px 35px rgba(0,0,0,0.15);
  color: white;
  text-align: center;
">
           <h2 style="font-size: 1.8rem; margin-bottom: 1.5rem;">Track, Analyze, and Optimize Your Study Habits</h2>
            <p style="font-size: 1.3rem; opacity: 0.9; margin-bottom: 2.5rem;">
                Reduce stress and boost academic performance with data-driven, personalized AI insights.
            </p>
        </div>
        """, unsafe_allow_html=True)

        st.markdown("<br>", unsafe_allow_html=True)

        # Prominent buttons
        btn_col1, btn_col2 = st.columns(2)
        with btn_col1:
            if st.button("Get Started →", use_container_width=True, type="primary", key="get_started_btn", help="Create a free account"):
                st.session_state.page = "signup"
                st.rerun()
        with btn_col2:
            if st.button("Login", use_container_width=True, key="login_btn"):
                st.session_state.page = "login"
                st.rerun()

    st.markdown("<br><br><br><br>", unsafe_allow_html=True)

    # Features Section
    st.markdown("<h2 style='text-align: center; color: #333;'>Why Choose StudyTrack AI?</h2>", unsafe_allow_html=True)
    st.markdown("<p style='text-align: center; color: #666; font-size: 1.2rem;'>Powerful features designed to help you study smarter, not harder.</p>", unsafe_allow_html=True)

    features = [
        ("📊", "Smart Dashboard", "Visualize study patterns, stress levels, and progress with beautiful interactive charts. Analyze your study patterns through visuals."),
        ("🤖", "AI Recommendations", "Receive tailored advice based on your unique habits and performance data. Get Personalized Recoomendation based on your habits"),
        ("📝", "Effortless Logging", "Log study hours, sleep, distractions, and exam scores in seconds. View your Log History"),
        ("📈", "Performance Insights", "Discover clear correlations between habits and grades.Improve your Habits based on Insights"),
    ]

    cols = st.columns(2)
    for idx, (icon, title, desc) in enumerate(features):
        with cols[idx % 2]:
            st.markdown(f"""
            <div style="background: white; height: 340px; padding: 2rem; border-radius: 20px; box-shadow: 0 8px 25px rgba(0,0,0,0.1); height: 100%; text-align: center; transition: transform 0.3s ease, box-shadow 0.3s ease; border: 1px solid #f0f0f0;">
                <div style="font-size: 3.5rem; margin-bottom: 1rem;">{icon}</div>
                <h3 style="color: #667eea; margin-bottom: 1rem; font-size: 1.4rem;">{title}</h3>
                <p style="color: #666; font-size: 1.2rem; line-height: 1.5;">{desc}</p>
            </div>
            <br>
            """, unsafe_allow_html=True)

    st.markdown("<br><br><br>", unsafe_allow_html=True)

    # Strong final CTA
    st.markdown("""
        <div style="
          max-width: 600px; width: 90%; height: 200px; padding: 3rem; margin: 0 auto;
          background: linear-gradient(135deg, #ad4b94 0%, #1A237E 100%);
          border-radius: 24px;
          box-shadow: 0 15px 35px rgba(0,0,0,0.15);
          color: white;
          text-align: center;">
          <h3 style="font-size: 1.8rem; margin-bottom: 1.5rem;">Ready to Transform your study habits </h3>
          <p style="font-size: 1.3rem; opacity: 0.9; margin-bottom: 2.5rem;">
                Join Study Track and start your journey now
            </p>
        </div>
        """, unsafe_allow_html=True)

    st.markdown("<br>", unsafe_allow_html=True)

    bottom_col1, bottom_col2, bottom_col3 = st.columns([1, 1, 1])
    with bottom_col2:
        if st.button("Get Started Now →", use_container_width=True, type="primary", key="bottom_get_started"):
            st.session_state.page = "signup"
            st.rerun()


# ---- Visualization Functions ----


def create_stress_gauge(df):
    """Creates a colorful gauge for average stress."""
    avg_stress = df["stress_level"].mean()

    fig = go.Figure(
        go.Indicator(
            mode="gauge+number",
            value=avg_stress,
            title={"text": "Average Stress Index (1-10)", "font": {"size": 18}},
            domain={"x": [0, 1], "y": [0, 1]},
            gauge={
                "axis": {"range": [0, 10], "tickwidth": 1, "tickcolor": "darkgray"},
                "bar": {"color": "#1abc9c"},
                "steps": [
                    {"range": [0, 3], "color": "#2ecc71"},
                    {"range": [3, 7], "color": "#f1c40f"},
                    {"range": [7, 10], "color": "#e74c3c"},
                ],
                "threshold": {
                    "line": {"color": "red", "width": 4},
                    "thickness": 0.75,
                    "value": 7,
                },
            },
        )
    )
    fig.update_layout(height=250, margin=dict(l=20, r=20, t=60, b=20))
    return fig


def create_study_trend_chart(df):
    """Creates a smooth line chart for study hours trend."""
    fig = px.line(
        df,
        x="date",
        y="study_hours",
        markers=True,
        title="Weekly Study Hours Trend",
        labels={"date": "Date", "study_hours": "Study Hours (h)"},
    )
    fig.update_traces(
        line=dict(color="#3498db", width=4),
        marker=dict(size=10, color="#f39c12", line=dict(width=1.5, color="white")),
    )
    fig.update_layout(
        xaxis_title=None,
        yaxis_title="Study Hours (h)",
        height=400,
        margin={"t": 30, "b": 10},
        plot_bgcolor="rgba(240,242,246,0.5)",
        paper_bgcolor="rgba(0,0,0,0)",
        font={"color": "#333333"},
    )
    return fig


def create_score_vs_effort_chart(df):
    """Creates a scatter plot for Exam Score vs Study Hours with trendline."""
    scatter_df = df[["study_hours", "exam_score"]].dropna()

    fig = px.scatter(
        scatter_df,
        x="study_hours",
        y="exam_score",
        trendline="ols",
        title="Exam Score vs. Study Hours (Correlation)",
        labels={"study_hours": "Study Hours (h)", "exam_score": "Exam Score (%)"},
        color_discrete_sequence=["#9b59b6"],
    )
    fig.update_traces(marker=dict(size=12, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
    fig.update_layout(
        height=400,
        margin={"t": 30, "b": 10},
        plot_bgcolor="rgba(240,242,246,0.5)",
        paper_bgcolor="rgba(0,0,0,0)",
        font={"color": "#333333"},
    )
    return fig


# ---- Reusable Log Form (Student & Admin CRUD) ----

def log_crud_form(user_id: Optional[int] = None, is_admin_mode: bool = False):
    """Reusable form for adding or updating a log."""

    # Check if a log is being updated
    log_id_to_update = st.session_state.get("admin_log_to_update")
    is_update = log_id_to_update is not None

    # Pre-populate data if updating
    initial_data = {}
    if is_update and is_admin_mode:
        initial_data = st.session_state.admin_log_data
        st.markdown(f"#### Update Log ID: **{log_id_to_update}** (Student: {initial_data.get('student_name', 'N/A')})")
    elif is_admin_mode:
        st.markdown("#### Admin: Add New Log (Feature not fully implemented)")
        st.warning("Admin 'Add Log' requires an input field for the target Student's ID/Email, which is currently skipped.")

    else:
        st.markdown("#### Record Your Session Details")

    # --- Form Implementation ---
    form_key = "update_log_form" if is_update else "add_log_form"

    # Use default values based on initial_data for update, or typical defaults for new log
    default_study = initial_data.get('study_hours', 4.0)
    default_sleep = initial_data.get('sleep_hours', 8.0)
    default_social = initial_data.get('social_media_hours', 1.0)
    default_stress = initial_data.get('stress_level', 5)
    default_exam_score = initial_data.get('exam_score') if initial_data.get('exam_score') is not None else None
    default_notes = initial_data.get('notes', "")

    # Convert exam score to float or None for the form value
    try:
        if default_exam_score is not None:
            default_exam_score = float(default_exam_score)
    except (ValueError, TypeError):
        default_exam_score = None


    with st.form(form_key, clear_on_submit=not is_update):
        col1, col2 = st.columns(2)
        with col1:
            study_h = st.number_input(
                "Study Hours (h)", min_value=0.0, max_value=20.0, value=default_study, step=0.5, key=f"study_h_{form_key}"
            )
            sleep_h = st.number_input(
                "Sleep Hours (h)", min_value=0.0, max_value=12.0, value=default_sleep, step=0.5, key=f"sleep_h_{form_key}"
            )
            social_h = st.number_input(
                "Social Media Hours (h)", min_value=0.0, max_value=15.0, value=default_social, step=0.5, key=f"social_h_{form_key}"
            )
        with col2:
            stress = st.slider("Stress Level",min_value=1,max_value=10,value=int(default_stress),step=1,key=f"stress_{form_key}")
            exam_s = st.number_input(
                "Exam/Quiz Score (%) ", min_value=35.0, max_value=100.0, step=0.5, format="%.1f",
                value=default_exam_score, placeholder="Enter score here (e.g., 85.5)", key=f"exam_s_{form_key}"
            )
            notes = st.text_area(
                "Notes & Reflections", placeholder="What topics were covered?", height=100,
                value=default_notes, key=f"notes_{form_key}"
            )

        submit_label = "Update Log" if is_update else " Save New Log"
        submitted = st.form_submit_button(submit_label, use_container_width=True, type="primary")

    if submitted:

        # === NEW: Validation for compulsory exam score ===
        if exam_s is None:
          st.error("Exam/Quiz Score is required. Please enter a score.")
          st.stop()
        payload = {
            "study_hours": float(study_h),
            "social_media_hours": float(social_h),
            "sleep_hours": float(sleep_h),
            "stress_level": int(stress),
            "exam_score": float(exam_s),
            "notes": notes.strip() if notes else None,
        }

        if is_update:
            # Update specific log using Admin endpoint
            payload["log_id"] = log_id_to_update
            endpoint = "/admin/update-study-log"
        else:
            # Student adding a log (uses existing /add-study-log endpoint)
            endpoint = "/add-study-log"

        # Execute the call
        with st.spinner(f"{submit_label}..."):
            resp = call(endpoint, payload)

        if is_update:
            if resp and "updated successfully" in (resp.get("message") or "").lower():
                st.success(f"Log ID {log_id_to_update} updated successfully!")
                # Clean up session state after update and rerender the Admin page
                st.session_state.pop("admin_log_to_update", None)
                st.session_state.pop("admin_log_data", None)
                # Set state to navigate back to All Logs
                st.session_state.admin_feature_key = "all_logs"
                st.rerun()
            else:
                st.error("Failed to update log. Error: " + (resp.get("detail") or resp.get("message") or "Unknown error."))
        else:
            # Student addition success
            if resp and resp.get("message") == "Study log added successfully":
                st.success("Log saved successfully! Dashboard insights updated.")
                st.balloons()
                # Rerun to clear the form and update dashboard metrics
                st.rerun()
            else:
                st.error("Failed to save log. Error: " + (resp.get("detail") or resp.get("message") or "Unknown error."))


# ---- Page Layout Functions ----


def show_login_page():
    st.title(" StudyTrack ")
    st.subheader("Your AI-Powered Study Companion")


    st.markdown("#### Welcome Back ")
    st.info("*Please log in to access your study dashboard or sign up to start tracking.*")

    st.markdown("---")


    col1, col2 = st.columns([1.5, 2])


    with col1:
        with st.form("login_form_main", clear_on_submit=False):
            email = st.text_input(" Email Address", placeholder="you@example.com")
            pwd = st.text_input(" Password", type="password")
            role = st.selectbox("Login as", ["student", "admin"])

            submitted = st.form_submit_button("Login", use_container_width=True, type="primary")


        if submitted:
            with st.spinner("Logging in..."):
                res = call("/login", {"email": email, "password": pwd, "role": role})

            if res and res.get("message") == "Login OK":
                st.session_state.user = res["user"]

                # --- CORRECTED REDIRECTION LOGIC ---
                if res["user"]["role"] == "admin":
                    st.session_state.page = "admin"  # Redirect Admin to their specific page
                else:
                    st.session_state.page = "dashboard" # Keep Students on the dashboard
                # -----------------------------------

                st.success(f"Welcome back, **{res['user']['name']}**! Redirecting...")
                st.rerun()
            else:
                st.error(res.get("detail") or res.get("message") or "Invalid credentials. Please check your email, password, and role.")


def show_signup_page():

    st.title("Start Your Journey to Better Focus")


    st.markdown("#### Create Student Account")
    st.info("*All new accounts are created as 'student' role.*")

    st.markdown("---")


    with st.form("signup_form", clear_on_submit=True): # Clear form upon success
        name = st.text_input(" Full Name", placeholder="John Doe")
        email = st.text_input(" Email Address", placeholder="john@example.com")
        pwd = st.text_input(" Choose Password", type="password", help="Use a strong password")

        submitted = st.form_submit_button("Create Account", use_container_width=True, type="primary")


    if submitted:
        with st.spinner("Creating account..."):
            # Role is hardcoded to student for general signup
            res = call("/signup", {"name": name, "email": email, "password": pwd, "role": "student"})

        if "successful" in (res.get("message") or "").lower():
            st.success("Account created successfully! You can now **log in**.")
            st.markdown("---")
            st.session_state.page = "login"
            st.rerun()
        else:
            st.error(res.get("detail") or res.get("message") or "Registration failed. This email may already be registered.")





# ---- Student Settings Page ----

def show_settings_page():
    st.markdown("## ⚙️ Profile & Settings")

    # Get current user data
    current_user = st.session_state.user
    user_id = current_user["id"]

    tab_profile, = st.tabs([" Profile Editing"])
    # ------------------ Tab 1: Profile Editing ------------------
    with tab_profile:
        st.markdown("#### Update Your Account Details")
        st.info(f"Current Email (Read-only): **{current_user['email']}**")

        # --- Form 1: Update Name ---
        with st.form("update_name_form"):
            new_name = st.text_input("Full Name", value=current_user.get("name", ""), placeholder="Enter your full name")
            name_submitted = st.form_submit_button("Update Name", type="primary")

            if name_submitted:
                if new_name.strip() and new_name.strip() != current_user.get("name"):
                    payload = {"new_name": new_name.strip()}  # ← Matches backend model
                    with st.spinner("Updating name..."):
                        resp = call("/update-name", payload)  # ← Correct endpoint

                    if resp and "success" in (resp.get("message") or "").lower():
                        st.session_state.user["name"] = new_name.strip()
                        st.success(f"Name updated successfully to **{new_name.strip()}**!")
                        st.rerun()
                    else:
                        st.error("Failed to update name. Error: " + (resp.get("detail") or resp.get("message") or "Unknown error"))
                else:
                    st.warning("Please enter a different name.")

        st.markdown("---")

        # --- Form 2: Update Password ---
        with st.form("reset_password_form"):
            new_pwd = st.text_input("New Password", type="password", key="new_pwd_input")
            confirm_pwd = st.text_input("Confirm New Password", type="password", key="confirm_pwd_input")
            submitted = st.form_submit_button("Reset Password", type="primary")


        if submitted:
            if new_pwd != confirm_pwd:
                st.error("Passwords do not match.")
            elif len(new_pwd) < 6: # Basic client-side validation
                st.error("Password must be at least 6 characters long.")
            else:
                payload = {
                    "user_id": user_id, # Use the validated ID
                    "new_password": new_pwd,
                }
                with st.spinner("Updating password..."):
                    res = call("/reset-password", payload)

                # --------- TOLERANT SUCCESS CHECK ----------
                success_msg = res.get("message") if isinstance(res, dict) else None
                if isinstance(success_msg, str):
                    lower = success_msg.lower()
                    # check for keywords rather than exact phrase
                    if ("password" in lower and ("updated" in lower or "success" in lower)) or "password updated successfully" in lower:
                        # success path
                        st.session_state.page = "login"
                        st.success("Password successfully reset! Please log in with your new password.")
                        # Final cleanup
                        st.session_state.pop("reset_stage", None)
                        st.session_state.pop("reset_email", None)
                        st.session_state.pop("reset_user_id", None)
                        st.rerun()
                        return
                # If we reach here, treat as error and show helpful backend response
                backend_msg = res.get("detail") or res.get("message") or "Unknown error."
                st.error("Failed to reset password. Error: " + str(backend_msg))




# ---- Student Dashboard (Settings Tab Removed) ----
def show_student_dashboard():
    user_id = st.session_state.user["id"]

    # --- New Settings Button (Top Right Placement Hack) ---
    col_header_title, col_header_settings = st.columns([0.85, 0.15])

    with col_header_title:
        st.title(f"Hello, {st.session_state.user['name'].split()[0]}! ")

    with col_header_settings:
        st.write("") # Spacer
        if st.button("Profile", key="dashboard_settings_btn", use_container_width=True):
            st.session_state.page = "settings" # Switch to the dedicated settings page
            st.rerun()

    st.subheader("Your Progress at a Glance")
    st.markdown("---")
    # --- End Settings Button ---


    # Fetch logs immediately for metrics and tabs
    logs = call(f"/study-logs/{user_id}")
    df_all = pd.DataFrame(logs) if logs and isinstance(logs, list) and "error" not in logs else None


    if logs and isinstance(logs, list) and len(logs) > 0 and "error" not in logs:

      df_recent = pd.DataFrame(logs)
      df_recent["date"] = pd.to_datetime(df_recent["date"])
      df_recent = df_recent.sort_values("date", ascending=False).head(10)  # Last 10 logs

      avg_study = df_recent["study_hours"].mean()
      avg_media = df_recent["social_media_hours"].mean()
      avg_stress = df_recent["stress_level"].mean()
      total_logs = len(df_all)

      col1, col2, col3, col4 = st.columns(4)
      col1.metric("Avg Daily Study Hours ", f"{avg_study:.1f} h")
      col2.metric("Avg Social Media Hours", f"{avg_media:.1f} h")
      col3.metric("Avg Stress Level", f"{avg_stress:.1f} / 10")
      col4.metric("Total", total_logs)


    # ------------------ Tabs for Navigation ------------------
    # Removed the " Settings" tab here!
    tab1, tab2, tab3 = st.tabs(
        [" Log Today's Progress", "Study History"," Personalized Recommendation "]
    )


    # ---- Tab 1: Logging ----
    with tab1:
        log_crud_form(user_id=user_id, is_admin_mode=False)

# ---- Tab 2: History ----
    with tab2:
        st.markdown("### Detailed Study Log History")
        if df_all is None or len(df_all) == 0:
            st.info("No logs yet. Add your first log in the 'Log Today's Progress' tab.")
        else:
            df = df_all.copy()
            df["date"] = safe_date_format(df["date"])
            df = df.sort_values("date", ascending=False)

            # Select relevant columns for the history table
            display_cols = [
                "date", "study_hours", "social_media_hours", "sleep_hours",
                "stress_level", "exam_score", "notes",
            ]

            st.dataframe(
                df[display_cols].rename(
                    columns={
                        "date": "Date & Time",
                        "study_hours": "Study (h)",
                        "social_media_hours": "Social Media (h)",
                        "sleep_hours": "Sleep (h)",
                        "stress_level": "Stress (1-10)",
                        "exam_score": "Score (%)",
                    }
                ),
                use_container_width=True,
                hide_index=True,
            )

    # ---- Tab 3: Visualizations ----
    # ---- Tab 3: Personalized Study Track + Visual Insights ----
    with tab3:
        # === AI Recommendation Section ===
        st.markdown("### Personalized Study Track")

        # Call your FastAPI backend
        rec = call(f"/recommendation/{st.session_state.user['id']}")

        if "error" in rec:
            st.error("Could not load recommendation. Is the backend running?")
            st.stop()

        # Welcome message for brand new users
        if rec.get("cluster_name") == "Welcome! Start Logging":
            st.info("Welcome! Log your first session to unlock your AI coach!")
            for tip in rec["tips"]:
                st.markdown(f"• {tip}")
            st.stop()

        # Extract data from API response
        cluster     = rec["cluster_name"]
        title       = rec["title"]
        emoji       = rec.get("emoji", "Student")
        tips        = rec["tips"]
        averages    = rec["averages"]
        logs_used   = rec["logs_used"]
        updated     = rec.get("updated_at", "Just now")
        desc        = rec.get("description", "Keep improving your habits!")

        # Beautiful gradient header
        st.markdown(f"""
        <div style="
            background: linear-gradient(135deg, #667eea 0%, #1A237E 100%);
            color: white;
            padding: 2rem;
            border-radius: 16px;
            margin: 1rem 0;
            text-align: center;
            box-shadow: 0 8px 32px rgba(102, 126, 234, 0.3);
        ">
            <h2 style="margin: 0; font-size: 2.2rem;"> {st.session_state.user['name'].split()[0]} {title}</h2>
            <p style="margin: 0.8rem 0 0; font-size: 1.1rem; opacity: 0.9;">
                Based on your logs • Updated {updated}
            </p>
        </div>
        """, unsafe_allow_html=True)

        # Short explanation
        st.info(f"**Why you're here:** {desc}")

        # === Visual Insights Section ===
        st.markdown("### Your Study Insights")
        st.markdown("*Discover patterns from your recent activity.*")

        user_id = st.session_state.user["id"]
        logs_response = call(f"/study-logs/{user_id}")

        if not logs_response or len(logs_response) == 0 or "error" in logs_response:
            st.info("No study logs yet! Start recording to unlock insights.")
            st.markdown("---")
            st.markdown("""
            **When you have data you’ll see:**
            - Study hours trend
            - Study vs exam score correlation
            - Sleep vs stress impact
            - Real-time stress gauge
            """)
        else:
            # Take only the last 10 logs
            df = pd.DataFrame(logs_response)
            df["date"] = pd.to_datetime(df["date"])
            df = df.sort_values("date", ascending=False).head(10).copy()
            df = df.sort_values("date").reset_index(drop=True)  # chronological for charts

            if len(df) < 2:
                st.info("Log at least 2 sessions to see meaningful charts.")
            else:
                df_scores = df.dropna(subset=["exam_score"])

                # 2×2 Grid of charts
                col1, col2 = st.columns(2)

                # 1. Study Hours Trend
                with col1:
                    st.markdown("#### Study Hours Trend")
                    fig_line = px.line(
                        df,
                        x="date",
                        y="study_hours",
                        markers=True,
                        title="Study Pattern (Last 10)",
                        color_discrete_sequence=["#3498db"]
                    )
                    fig_line.update_traces(line=dict(width=4), marker=dict(size=9))
                    fig_line.update_layout(height=320, xaxis_title=None, yaxis_title="Hours")
                    st.plotly_chart(fig_line, use_container_width=True)

                # 2. Study vs Exam Score
                with col2:
                    st.markdown("#### Study vs Exam Score")
                    if len(df_scores) >= 2:
                        fig_scatter = px.scatter(
                            df_scores,
                            x="study_hours",
                            y="exam_score",
                            trendline="ols",
                            trendline_color_override="#e74c3c",
                            color_discrete_sequence=["#9b59b6"]
                        )
                        fig_scatter.update_traces(marker=dict(size=11, opacity=0.9))
                        fig_scatter.update_layout(height=320, xaxis_title="Study Hours", yaxis_title="Score (%)")
                        st.plotly_chart(fig_scatter, use_container_width=True)
                    else:
                        st.info("Need 2+ exam scores")

                # Second row
                col3, col4 = st.columns(2)

                # 3. Sleep vs Stress
                with col3:
                    st.markdown("#### Sleep vs Stress")
                    fig_sleep = px.scatter(
                        df,
                        x="sleep_hours",
                        y="stress_level",
                        size="study_hours",
                        color="stress_level",
                        color_continuous_scale="RdYlGn_r",
                    )
                    fig_sleep.update_layout(height=320, xaxis_title="Sleep Hours", yaxis_title="Stress (1–10)")
                    st.plotly_chart(fig_sleep, use_container_width=True)

                # 4. Stress Gauge
                with col4:
                    st.markdown("#### Average Stress Level")
                    avg_stress = round(df["stress_level"].mean(), 1)

                    fig_gauge = go.Figure(go.Indicator(
                        mode="gauge+number",
                        value=avg_stress,
                        domain={"x": [0, 1], "y": [0, 1]},
                        gauge={
                            "axis": {"range": [None, 10]},
                            "bar": {"color": "#FF4B4B"},
                            "bgcolor": "white",
                            "borderwidth": 2,
                            "bordercolor": "gray",
                            "steps": [
                                {"range": [0, 4], "color": "#d5f5e3"},
                                {"range": [4, 7], "color": "#fef9e7"},
                                {"range": [7, 10], "color": "#fadbd8"}
                            ],
                            "threshold": {
                                "line": {"color": "red", "width": 6},
                                "thickness": 0.8,
                                "value": 7
                            }
                        }
                    ))

                    fig_gauge.update_layout(height=320, margin=dict(t=70, b=20, l=20, r=20))
                    st.plotly_chart(fig_gauge, use_container_width=True)

                # === Action Plan (below all charts) ===
                st.markdown("### Your Personalized Action Plan")

                colors = [
                    "#4299e1",  # Blue
                    "#48bb78",  # Green
                    "#ed8936",  # Orange
                    "#9f7aea",  # Purple
                    "#f56565",  # Red
                ]

                bg_colors = [
                    "#2596be",
                    "#f7fafc",
                ]

                for idx, tip in enumerate(tips):
                    border_color = colors[idx % len(colors)]
                    bg_color = bg_colors[idx % len(bg_colors)]

                    st.markdown(
                        f"""
                        <div style="
                            background: {bg_color};
                            color: #2d3748;
                            padding: 1.6rem 2rem;
                            border-radius: 18px;
                            border-left: 8px solid {border_color};
                            margin: 1.2rem 0;
                            font-size: 1.3rem;
                            box-shadow: 0 6px 24px rgba(0,0,0,0.08);
                            transition: all 0.3s ease;
                            font-weight: 500;
                            line-height: 1.6;
                        "
                        onmouseover="
                            this.style.transform='translateY(-6px)';
                            this.style.boxShadow='0 16px 36px rgba(0,0,0,0.15)';
                        "
                        onmouseout="
                            this.style.transform='translateY(0)';
                            this.style.boxShadow='0 6px 24px rgba(0,0,0,0.08)';
                        ">
                            {tip}
                        </div>
                        """,
                        unsafe_allow_html=True
                    )






# ---- Admin Dashboard (Enhanced with Metrics & Charts) ----
def show_admin_dashboard():
    st.title("Dashboard")
    st.subheader("System Overview & Student Management")

    # Fetch core data once
    with st.spinner("Loading system data..."):
        students_data = call("/students")
        all_logs_data = call("/admin/all-study-logs")

    # Handle errors gracefully
    if not students_data or "error" in students_data:
        st.error("Unable to load student data.")
        students_data = []
    if not all_logs_data or "error" in all_logs_data:
        st.warning("Unable to load full log data for charts/metrics. Some features limited.")
        all_logs_data = []

    df_students = pd.DataFrame(students_data)
    df_logs = pd.DataFrame(all_logs_data)

    total_students = len(df_students)
    total_logs = len(df_logs)
    avg_study_hours = df_logs["study_hours"].mean() if not df_logs.empty else 0
    avg_stress = df_logs["stress_level"].mean() if not df_logs.empty else 0

    # --- Key Metrics Row ---
    st.markdown("### System-Wide Metrics")
    col1, col2, col3, col4 = st.columns(4)
    col1.metric("Total Registered Students", total_students)
    col2.metric("Total Study Logs", total_logs)
    col3.metric("Avg Study Hours (All Time)", f"{avg_study_hours:.1f} h" if avg_study_hours else "N/A")
    col4.metric("Avg Stress Level", f"{avg_stress:.1f}/10" if avg_stress else "N/A")

    st.markdown("---")

    # 1. Map state keys to displayed names
    tab_map = {
        "overview": "Students Overview ",
        "add_user": "Add New User ",
        "all_logs": "All Logs ",
        "crud": "Log CRUD Operations "
    }

    # Get the current selected key or default to "overview"
    selected_key = st.session_state.get("admin_feature_key", "overview")

    # Place navigation buttons in the sidebar
    with st.sidebar:
        st.header("Admin Features")

        # Create buttons for each feature using a loop
        for key, name in tab_map.items():
            is_active = (key == selected_key)

            # Use a unique key and set type='primary' for the active button
            button_style = "primary" if is_active else "secondary"

            if st.button(
                name,
                key=f"admin_nav_{key}",
                use_container_width=True,
                type=button_style
            ):
                st.session_state.admin_feature_key = key

                # If clicking a new feature, clear any pending CRUD actions
                if key != "crud":
                    st.session_state.pop("admin_log_to_update", None)
                    st.session_state.pop("admin_log_data", None)

                st.rerun() # RERUN to show the correct content

    st.markdown("### AI Student Behavior Clusters")

    # Fast & accurate cluster data from new endpoint
    cluster_resp = call("/admin/clusters")

    if cluster_resp and "distribution" in cluster_resp:
        dist = cluster_resp["distribution"]
        names = cluster_resp["names"]
        total = cluster_resp["total_clustered"]

        if total == 0:
            st.info("No students have been clustered yet. Clusters appear after students log study sessions.")
        else:
            # Prepare data for pie chart
            labels = [names[cid] for cid, count in dist.items() if count > 0]
            values = [count for count in dist.values() if count > 0]

            colors = {
                "Average": "#95a5a6",
                "Stressed Over-Achiever": "#e74c3c",
                "Balanced Top Performer": "#27ae60",
                "Distracted": "#f39c12"
            }
            pie_colors = [colors[name] for name in labels]

            # Donut Pie Chart
            fig = go.Figure(data=[go.Pie(
                labels=labels,
                values=values,
                hole=0.45,
                marker=dict(colors=pie_colors, line=dict(color='#ffffff', width=3)),
                textinfo='label+percent',
                textposition='inside',
                hoverinfo='label+value+percent',
                showlegend=False
            )])

            fig.update_layout(
                title=dict(text="Student Cluster Distribution", x=0.5, font=dict(size=20)),
                height=420,
                margin=dict(t=100, b=40, l=40, r=40),
                annotations=[dict(
                    text=f'{total}<br>Students',
                    font_size=20,
                    showarrow=False
                )]
            )

            st.plotly_chart(fig, use_container_width=True)

            # Summary Cards
            st.markdown("#### Breakdown by Cluster")
            cols = st.columns(4)
            for i, (cid, count) in enumerate(dist.items()):
                name = names[cid]
                color = colors[name]
                percentage = (count / total * 100) if total > 0 else 0

                with cols[i]:
                    st.markdown(f"""
                    <div style="
                        background: {color}15;
                        padding: 1.4rem;
                        border-radius: 16px;
                        text-align: center;
                        border-left: 8px solid {color};
                        box-shadow: 0 6px 20px rgba(0,0,0,0.08);
                        height: 140px;
                    ">
                        <h4 style="color: {color}; margin: 0; font-size: 1.1rem;">{name}</h4>
                        <h2 style="margin: 0.8rem 0 0; color: #2c3e50;">{count}</h2>
                        <p style="margin: 0; color: #7f8c8d; font-size: 0.95rem;">
                            {percentage:.1f}% of students
                        </p>
                    </div>
                    """, unsafe_allow_html=True)

            st.markdown("---")
    else:
        st.warning("Could not load cluster data. Is the backend running and is the user_clusters table populated?")
    # ------------------ Tab 1: Students Overview (selected_key == "overview") ------------------
    if selected_key == "overview":
      st.markdown("#### Registered Students")

      with st.spinner("Fetching student data..."):
          data = call("/students")

      if not data or "error" in data:
          st.error("No students found or unable to load data. Error: " + (data.get("detail") or str(data)))
      else:
          df_students = pd.DataFrame(data)

          st.dataframe(
              df_students[["id", "name", "email"]],
              use_container_width=True,
              hide_index=True,
              column_config={
                  "id": st.column_config.TextColumn("ID", width="small"),
                  "name": "Student Name",
                  "email": "Email"
              },
          )

          st.markdown("---")
          st.markdown("#### Student Logs & Cluster")

          # Dropdown to select student
          student_map = {f"{row['name']} ({row['email']})": row['id'] for index, row in df_students.iterrows()}
          choice_list = ["-- Select a student --"] + list(student_map.keys())
          choice = st.selectbox("Select a student to inspect:", choice_list, key="overview_student_select")

          if choice != "-- Select a student --":
              selected_id = student_map[choice]

              # Two columns layout for clean display
              col1, col2 = st.columns([1, 1])

              with col1:
                  with st.spinner(f"Fetching study logs for {choice}..."):
                      logs = call(f"/study-logs/{selected_id}")

                  if logs and len(logs) > 0 and "error" not in logs:
                      log_df = pd.DataFrame(logs)
                      log_df["date"] = safe_date_format(log_df["date"])
                      st.markdown(f"##### Study History for {choice}")

                      st.dataframe(
                          log_df[
                              [
                                  "date", "study_hours", "social_media_hours", "sleep_hours",
                                  "stress_level", "exam_score", "notes",
                              ]
                          ].rename(
                              columns={
                                  "date": "Date", "study_hours": "Study (h)",
                                  "social_media_hours": "Social Media (h)", "sleep_hours": "Sleep (h)",
                                  "stress_level": "Stress (1-10)", "exam_score": "Score (%)",
                              }
                          ),
                          use_container_width=True,
                          hide_index=True,
                      )
                  else:
                      st.info(f"{choice} has no study logs recorded yet.")

              with col2:
                  with st.spinner(f"Fetching cluster info for {choice}..."):
                      cluster_data = call(f"/student-cluster/{selected_id}")

                  if "error" in cluster_data:
                      st.error("Failed to load cluster info: " + cluster_data.get("detail", "Unknown error"))
                  else:
                      st.markdown(f"##### Cluster Details for {choice}")
                      st.markdown(f"**User ID:** {cluster_data['user_id']}")
                      st.markdown(f"**Name:** {cluster_data['name']}")

                      if cluster_data["cluster_id"] is None:
                          st.warning("No cluster assigned yet – student needs at least one study log!")
                      else:
                          st.success(f"**Cluster ID:** {cluster_data['cluster_id']}")
                          st.markdown(f"**Cluster Name:** {cluster_data['cluster_name']}")



    # ------------------ Tab 2: Add New User (selected_key == "add_user") ------------------
    elif selected_key == "add_user":
        st.markdown("#### Add New System User (Student or Admin)")
        with st.form("add_user_form", clear_on_submit=True):
            n = st.text_input("Full Name", placeholder="New User Name")
            e = st.text_input("Email Address", placeholder="newuser@system.com")
            p = st.text_input("Password", type="password")
            r = st.selectbox("User Role", ["student", "admin"])
            submitted = st.form_submit_button(" Add User", use_container_width=True, type="primary")

        if submitted:
            with st.spinner(f"Adding user {e}..."):
                resp = call("/add-user", {"name": n, "email": e, "password": p, "role": r})

            if resp and "added" in (resp.get("message") or "").lower():
                # FIX: Removed st.rerun() to allow the success message to be seen
                st.success(f"User **{n}** ({e}) added successfully as **{r.upper()}**! The form fields have been cleared.")
            else:
                st.error(resp.get("detail") or resp.get("message") or "Failed to add user. Check if email already exists.")


    # ------------------ Tab 3: All Study Logs (selected_key == "all_logs") ------------------
    elif selected_key == "all_logs":
        st.markdown("#### Comprehensive Study Logs (All Students)")

        with st.spinner("Fetching all study logs..."):
            all_logs = call("/admin/all-study-logs")

        if not all_logs or "error" in all_logs:
            if all_logs.get("detail") == "Access denied. Admin privileges required.":
                 st.error("Admin Access Required: The backend denied access to this endpoint.")
            else:
                 st.info("No study logs found across the system.")
                 st.error("Error fetching logs: " + (all_logs.get("detail") or str(all_logs)))
        else:
            df_logs = pd.DataFrame(all_logs)
            df_logs["date"] = safe_date_format(df_logs["date"])

            display_cols = [
                "log_id", "date", "student_name", "student_email", "study_hours", "social_media_hours",
                "sleep_hours", "stress_level", "exam_score", "notes"
            ]

            st.dataframe(
                df_logs[display_cols].rename(
                    columns={
                        "log_id": "Log ID",
                        "date": "Date & Time",
                        "student_name": "Student",
                        "student_email": "Email",
                        "study_hours": "Study (h)",
                        "social_media_hours": "Social Media (h)",
                        "sleep_hours": "Sleep (h)",
                        "stress_level": "Stress (1-10)",
                        "exam_score": "Score (%)",
                    }
                ),
                use_container_width=True,
                hide_index=True,
                column_config={"Log ID": st.column_config.TextColumn("Log ID", width="small")},
            )

            total_logs = len(df_logs)
            avg_study = df_logs["study_hours"].mean()
            colA, colB = st.columns(2)
            colA.metric("Total System Logs", total_logs)
            colB.metric("Avg Study Hours (System)", f"{avg_study:.2f} h")

            # --- Log Actions ---
            st.markdown("---")
            st.markdown("#### Select Log for Quick Action")
            col_actions = st.columns(3)

            with col_actions[0]:
                min_id = df_logs["log_id"].min() if not df_logs.empty else 1
                max_id = df_logs["log_id"].max() if not df_logs.empty else 1
                log_id_to_act = st.number_input("Log ID", min_value=min_id, max_value=max_id, step=1, key="action_log_id")

            with col_actions[1]:
                if st.button(" Edit Log", key="edit_log_btn", use_container_width=True):
                    if log_id_to_act and log_id_to_act in df_logs["log_id"].values:
                        selected_log = df_logs[df_logs["log_id"] == log_id_to_act].iloc[0].to_dict()

                        st.session_state.admin_log_to_update = log_id_to_act
                        st.session_state.admin_log_data = selected_log

                        # Set state to force switch to the 'crud' option on next run
                        st.session_state.admin_feature_key = "crud"

                        st.rerun()
                    else:
                        st.error(f"Log ID {log_id_to_act} not found in the table.")

            with col_actions[2]:
                if st.button(" Delete Log", key="delete_log_btn", use_container_width=True):
                    with st.spinner(f"Deleting Log ID {log_id_to_act}..."):
                        resp = call("/admin/delete-study-log", {"log_id": int(log_id_to_act)})

                    if resp and "deleted successfully" in (resp.get("message") or "").lower():
                        st.success(f"Successfully deleted Log ID {log_id_to_act}.")
                        st.rerun()
                    else:
                        st.error("Failed to delete log. Error: " + (resp.get("detail") or str(resp)))


    # ------------------ Tab 4: CRUD Operations (selected_key == "crud") ------------------
    elif selected_key == "crud":
        st.markdown("### Log Management (Update & Delete)")

        # 1. Update Form Stage: Only visible if a log ID is in state
        if st.session_state.get("admin_log_to_update"):
            log_crud_form(is_admin_mode=True)
            if st.button("Cancel Update", key="cancel_update"):
                st.session_state.pop("admin_log_to_update", None)
                st.session_state.pop("admin_log_data", None)
                # Reset feature key back to 'all_logs' after cancel
                st.session_state.admin_feature_key = "all_logs"
                st.rerun()
        else:
            # 2. Default View (Instruction and Delete Form)
            st.info("To modify a log, use the **'All Logs '** feature in the sidebar to select the entry first. Use the form below for immediate deletion by ID.")

            st.markdown("---")
            st.markdown("#### Delete Log by ID")

            with st.form("delete_log_form"):
                delete_id = st.number_input("Log ID to Delete", min_value=1, step=1, key="delete_id_input")
                delete_submitted = st.form_submit_button("Confirm Delete", type="secondary")

            if delete_submitted:
                if st.session_state.get("confirm_delete_id") != delete_id:
                    st.session_state.confirm_delete_id = delete_id
                    st.warning(f"Are you sure you want to delete Log ID **{delete_id}**? Click 'Confirm Delete' again to finalize.")
                else:
                    with st.spinner(f"Deleting Log ID {delete_id}..."):
                        resp = call("/admin/delete-study-log", {"log_id": int(delete_id)})

                    if resp and "deleted successfully" in (resp.get("message") or "").lower():
                        st.success(f"Successfully deleted Log ID {delete_id}.")
                        st.session_state.pop("confirm_delete_id", None)
                        st.rerun()
                    else:
                        st.error("Failed to delete log. Error: " + (resp.get("detail") or str(resp)))




# ---- Main App Logic (Updated Routing) ----

# ---- Main App Logic (Updated Routing) ----
def main_app():
    # ---- Sidebar Navigation & Auth ----
    with st.sidebar:
        if st.session_state.user:
            role = st.session_state.user["role"].title()
            name = st.session_state.user["name"]
            st.success(f"**{role}**: {name}")

            if role == "Student":
                st.header("Student Panel")
                if st.button("Student", use_container_width=True):
                    st.session_state.page = "dashboard"
                    st.rerun()
            elif role == "Admin":
                st.header("Admin Menu")
                if st.button("Dashboard", use_container_width=True):
                    st.session_state.page = "admin"
                    st.rerun()

            st.markdown("---")
            if st.button("Logout", use_container_width=True):
                st.session_state.user = None
                st.session_state.page = "welcome"
                st.session_state.pop("admin_log_to_update", None)
                st.session_state.pop("admin_feature_key", None)
                st.success("Logged out successfully.")
                st.rerun()
        else:
            st.header("Welcome")
            if st.session_state.page != "welcome":
                if st.button("← Home page", use_container_width=True):
                    st.session_state.page = "welcome"
                    st.rerun()

            if st.session_state.page == "login":
                if st.button("New User? Sign Up", use_container_width=True):
                    st.session_state.page = "signup"
                    st.rerun()
            elif st.session_state.page == "signup":
                if st.button("Already have an account? Login", use_container_width=True):
                    st.session_state.page = "login"
                    st.rerun()

    # ---- Page Routing ----
    if st.session_state.user is None:
        if st.session_state.page == "welcome":
            show_welcome_page()
        elif st.session_state.page == "signup":
            show_signup_page()
        else:
            show_login_page()
    else:
        if st.session_state.user["role"] == "admin":
            show_admin_dashboard()
        elif st.session_state.user["role"] == "student":
            if st.session_state.page == "settings":
                show_settings_page()
                if st.button("← Back to Dashboard"):
                    st.session_state.page = "dashboard"
                    st.rerun()
            else:
                show_student_dashboard()


st.markdown("""
<style>
/* Full page background */
html, body, [data-testid="stAppViewContainer"] {
    background: linear-gradient(
        180deg,
        #FFFFFF 0%,
        #B2DFDB 100%
    ) !important;
}

/* Main content background transparent */
[data-testid="stAppViewContainer"] > .main {
    background: transparent !important;
}

</style>
""", unsafe_allow_html=True)


if __name__ == "__main__":
    main_app()

Writing frontend.py


In [13]:

!streamlit run frontend.py --server.port 8501 --server.headless true &> logs.txt &
from pyngrok import ngrok
import time
time.sleep(8)
url = ngrok.connect(8501, bind_tls=True)
print("STREAMLIT PUBLIC URL:")
print(url.public_url)

STREAMLIT PUBLIC URL:
https://a646-34-26-222-18.ngrok-free.app
